In [1]:
import concurrent.futures as futures
import multiprocessing
import os
import re

import pickle
import numpy as np
from pytorch_lightning import (
    LightningDataModule,
    LightningModule,
    Trainer,
    seed_everything,
)
from pytorch_lightning.callbacks import ModelCheckpoint

import torch
import datasets

from torch.utils.data import DataLoader
from transformers import (
    # AdamW,  # this does not work
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

from torchmetrics.classification import BinaryAccuracy, BinaryF1Score

/home/ant/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 0
rng = np.random.default_rng(SEED)
GEN_SEED = torch.Generator().manual_seed(SEED)
seed_everything(SEED, workers=True)
MODEL_NAME = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
DATASET_NAME = "dataset_def.pkl"

os.environ["TOKENIZERS_PARALLELISM"] = "true"

Global seed set to 0


In [5]:
with open(f"../../{DATASET_NAME}", "rb") as f:
    tuple_dataset = pickle.load(f)

print("loaded dataset")
print("dataset: ", len(tuple_dataset))
print(tuple_dataset[1])

loaded dataset
dataset:  62359
('patientregistry idcenter 160 sex F yeardiagnosisdiabetes 2002-01-01 levelofeducation [UNK] maritalstatus [UNK] profession [UNK] yearofbirth 1938-01-01 yearfirstaccess 2002-01-01 yearofdeath [UNK] diagnosis date 2014-10-21 amdcode AMD245 meaning Foot malformations value 736.7 date 2014-10-21 amdcode AMD302 meaning Not diabetic nephropathy value S date 2014-10-21 amdcode AMD247 meaning Other comorbidities value 736.70 date 2014-10-21 amdcode AMD129 meaning Absent diabetic neuropathy value S date 2014-10-21 amdcode AMD058 meaning Not diabetic foot value S date 2013-02-22 amdcode AMD302 meaning Not diabetic nephropathy value S date 2012-06-27 amdcode AMD302 meaning Not diabetic nephropathy value S date 2011-10-31 amdcode AMD302 meaning Not diabetic nephropathy value S date 2011-04-04 amdcode AMD302 meaning Not diabetic nephropathy value S date 2010-09-06 amdcode AMD302 meaning Not diabetic nephropathy value S date 2010-03-09 amdcode AMD302 meaning Not diabe

In [ ]:
# class PubMedBERTDataset(Dataset):
#     def __init__(self, data):
#         # here data is a list of tuples,
#         # each containing the patient history string and their label
#         self.data = data

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         patient_history = self.data[idx][0]
#         label = self.data[idx][1]
#         return patient_history, label

In [6]:
def convert_to_huggingfaceDataset(tuple_dataset):
    # here data is a list of tuples,
    # each containing the patient history string and their label
    # we need to convert it to a hugginface dataset
    dict_list = [{"label": data[1], "text": data[0]} for data in tuple_dataset]
    dataset = datasets.Dataset.from_list(dict_list)
    return dataset

In [7]:
max_len = [0] * 100
index = [0] * 100

# Save the 100 most length sentence and their indexes
for i, (ph, l) in enumerate(tuple_dataset):
    # Count the number of words in the ph variable
    # if CLEANED:
    ph_len = len(ph.split(" "))
    # else:
    #     ph_len = len(ph.replace(" ", "=").split("="))
    for j, x in enumerate(max_len):
        if ph_len > x:
            max_len.insert(j, ph_len)
            index.insert(j, i)
            max_len.pop()
            index.pop()
            break

print(max_len)
print(index)

[24413, 23724, 22705, 21974, 21598, 20658, 20097, 19994, 19903, 19872, 19715, 19320, 19204, 19075, 18900, 18867, 18848, 18686, 18654, 18641, 18531, 18530, 18526, 18512, 18319, 18314, 18283, 18023, 17955, 17931, 17921, 17890, 17812, 17765, 17746, 17677, 17566, 17546, 17517, 17471, 17462, 17460, 17445, 17298, 17273, 17249, 17244, 17208, 17158, 17060, 17047, 16998, 16995, 16972, 16967, 16938, 16931, 16873, 16821, 16748, 16719, 16661, 16646, 16639, 16589, 16498, 16402, 16398, 16382, 16373, 16291, 16230, 16211, 16207, 16165, 16143, 16076, 15967, 15955, 15939, 15927, 15926, 15922, 15921, 15918, 15871, 15852, 15816, 15808, 15804, 15800, 15796, 15785, 15768, 15747, 15743, 15743, 15671, 15663, 15637]
[44935, 24048, 10733, 33131, 28682, 32314, 42120, 14048, 20569, 26580, 14513, 11938, 23535, 21177, 27953, 37618, 19087, 44231, 33664, 8988, 21314, 17241, 24822, 22679, 6138, 43419, 4210, 4040, 29251, 8602, 29885, 9248, 34670, 12760, 12546, 5086, 19920, 44460, 39165, 38755, 24341, 1717, 16304, 29436

In [8]:
class PubMedBERTDataModule(LightningDataModule):
    def __init__(
        self,
        tuple_dataset,
        model_name_with_path: str,
        max_seq_length: int = 512,  # 512 is the max length of BERT and PubMedBERT but I need 32768 for not CLEANED and 24413 for CLEANED
        train_batch_size: int = 8,
        eval_batch_size: int = 8,
        **kwargs,
    ):
        super().__init__()
        self.model_name_with_path = model_name_with_path
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name_with_path, use_fast=True
        )

    def setup(self, stage=None):
        dataset = convert_to_huggingfaceDataset(tuple_dataset)
        tokenized_dataset = dataset.map(
            self.convert_to_features,
            batched=True,
            remove_columns=["text", "label"],
        )
        tokenized_dataset.set_format(type="torch")

        # split dataset into train and validation sampling randomly
        # use 20% of training data for validation
        train_set_size = int(len(tokenized_dataset) * 0.8)
        valid_set_size = len(tokenized_dataset) - train_set_size

        # split the dataset randomly into two
        self.train_data, self.valid_data = torch.utils.data.random_split(
            tokenized_dataset, [train_set_size, valid_set_size], generator=GEN_SEED
        )

    def prepare_data(self):
        AutoTokenizer.from_pretrained(
            self.model_name_with_path,
            use_fast=True,
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.train_batch_size,
            shuffle=True,
            num_workers=8,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid_data,
            batch_size=self.eval_batch_size,
            shuffle=False,
            num_workers=4,
        )

    def test_dataloader(self):
        # placeholder
        return DataLoader(
            self.valid_data,
            batch_size=self.eval_batch_size,
            shuffle=False,
            num_workers=4,
        )

    def convert_to_features(self, example_batch, indices=None):
        # Tokenize the patient history
        features = self.tokenizer(
            text=example_batch["text"],
            max_length=self.max_seq_length,
            padding="longest",
            truncation=True,
            return_tensors="pt",
        )
        # Rename label to labels to make it easier to pass to model forward
        features["labels"] = example_batch["label"]

        return features

In [7]:
# dm = PubMedBERTDataModule(tuple_dataset, MODEL_NAME)
# dm.prepare_data()
# dm.setup("fit")
# next(iter(dm.train_dataloader()))

In [9]:
class PubMedBERTTransformer(LightningModule):
    def __init__(
        self,
        model_name_or_path: str,
        num_labels: int = 2,  # It will create a PubMedBERT model (in our case) instance with encoder weights copied from the PubMedBERT model and a randomly initialized sequence classification head on top of the encoder with an output size of 2
        learning_rate: float = 2e-5,
        adam_epsilon: float = 1e-8,
        warmup_steps: int = 0,
        weight_decay: float = 0.0,
        **kwargs,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.config = AutoConfig.from_pretrained(
            model_name_or_path, num_labels=num_labels
        )
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name_or_path, config=self.config
        )
        self.train_acc_metric = BinaryAccuracy()
        self.val_acc_metric = BinaryAccuracy()
        self.train_f1_metric = BinaryF1Score()
        self.val_f1_metric = BinaryF1Score()

    def forward(self, **inputs):
        return self.model(**inputs)

    def step(self, batch):
        outputs = self(**batch)
        loss, logits = outputs[:2]
        if self.hparams.num_labels > 1:
            preds = logits.argmax(axis=1)
        elif self.hparams.num_labels == 1:
            preds = logits.squeeze()
        labels = batch["labels"]
        return {"loss": loss, "logits": logits, "preds": preds, "labels": labels}

    def training_step(self, batch, batch_idx):
        outputs = self.step(batch)
        self.train_acc_metric(outputs["preds"], outputs["labels"])
        self.train_f1_metric(outputs["preds"], outputs["labels"])
        self.log(
            "train_acc",
            self.train_acc_metric,
            on_step=True,
            on_epoch=True,
            prog_bar=False,
        )
        self.log(
            "train_f1", self.train_f1_metric, on_step=True, on_epoch=True, prog_bar=True
        )
        self.log(
            "train_loss", outputs["loss"], on_step=True, on_epoch=True, prog_bar=True
        )
        return outputs["loss"]

    def validation_step(self, batch, batch_idx):
        outputs = self.step(batch)
        self.val_acc_metric(outputs["preds"], outputs["labels"])
        self.val_f1_metric(outputs["preds"], outputs["labels"])
        self.log("val_acc", self.val_acc_metric, on_epoch=True, prog_bar=True)
        self.log("val_f1", self.val_f1_metric, on_epoch=True, prog_bar=True)
        self.log("val_loss", outputs["loss"], on_epoch=True, prog_bar=True)
        return {
            "loss": outputs["loss"],
            "preds": outputs["preds"],
            "labels": outputs["labels"],
        }

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if not any(nd in n for nd in no_decay)
                ],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [
                    p
                    for n, p in model.named_parameters()
                    if any(nd in n for nd in no_decay)
                ],
                "weight_decay": 0.0,
            },
        ]
        optimizer = torch.optim.AdamW(
            optimizer_grouped_parameters,
            lr=self.hparams.learning_rate,
            eps=self.hparams.adam_epsilon,
        )

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.hparams.warmup_steps,
            num_training_steps=self.trainer.estimated_stepping_batches,
        )
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

In [9]:
dm = PubMedBERTDataModule(tuple_dataset, MODEL_NAME)
dm.setup("fit")
# print(next(iter(dm.train_dataloader())))

model = PubMedBERTTransformer(
    model_name_or_path=MODEL_NAME,
)

trainer = Trainer(
    max_epochs=2,
    accelerator="auto",
    devices="auto",
)
trainer.fit(model=model, datamodule=dm)

Parameter 'function'=<bound method PubMedBERTDataModule.convert_to_features of <__main__.PubMedBERTDataModule object at 0x7f7f597c0d10>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 150/150 [00:00<00:00, 187.21 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ant/miniconda3/envs/ml/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: 100%|██████████| 30/30 [00:23<00:00,  1.29it/s, v_num=0, train_loss=0.0398]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 30/30 [00:25<00:00,  1.18it/s, v_num=0, train_loss=0.0398]


In [32]:
patient_index = 10733

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(tuple_dataset[patient_index][0])
inputs = tokenizer(
    tuple_dataset[patient_index][0],
    max_length=512,
    truncation=True,
    return_tensors="pt",
).to("cuda" if torch.cuda.is_available() else "cpu")
print(tuple_dataset[patient_index][1])

patientregistry idcenter 140 sex M yeardiagnosisdiabetes 1989-01-01 levelofeducation [UNK] maritalstatus 2.0 profession [UNK] yearofbirth 1960-01-01 yearfirstaccess 2012-01-01 yearofdeath [UNK] diagnosis date 2019-08-03 amdcode AMD245 meaning Foot malformations value 736.7 date 2019-08-03 amdcode AMD247 meaning Other comorbidities value 736.70 date 2019-08-03 amdcode AMD060 meaning Previous ulcer value 707.1 date 2019-08-03 amdcode AMD059 meaning Trophic lesion value 707.1 date 2019-08-03 amdcode AMD247 meaning Other comorbidities value 707.1 date 2019-07-13 amdcode AMD245 meaning Foot malformations value 736.7 date 2019-07-13 amdcode AMD060 meaning Previous ulcer value 707.1 date 2019-07-13 amdcode AMD247 meaning Other comorbidities value 707.1 date 2019-07-13 amdcode AMD247 meaning Other comorbidities value 736.70 date 2019-07-13 amdcode AMD059 meaning Trophic lesion value 707.1 date 2019-06-15 amdcode AMD247 meaning Other comorbidities value 707.1 date 2019-06-15 amdcode AMD247 mean

In [33]:
# this doesn't work! it output random predictions
# config = AutoConfig.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(
#     "../../lightning_logs/version_9/checkpoints/epoch=2-step=18708.ckpt", config=config
# ).to("cuda" if torch.cuda.is_available() else "cpu")
# with torch.no_grad():
#     logits = model(**inputs).logits
#     predicted_class = logits.argmax(axis=1).item()
#     print("Predicted class:", predicted_class)

In [36]:
model = PubMedBERTTransformer(
    model_name_or_path=MODEL_NAME,
).load_from_checkpoint(
    "../../lightning_logs/version_9/checkpoints/epoch=2-step=18708.ckpt"
)

# disable randomness, dropout, etc...
model.eval()

# predict with the model
y_hat = model(**inputs).logits.argmax(axis=1).item()
print("Predicted class:", y_hat)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 1
